## Let's do some real estate data-sourcing

We have 3 basic data sources:

[Merged Sales](data/zip/merged_sales_stats.csv) - This is sales data by zip code for California. It came from https://www.redfin.com/news/data-center/ and it's got a lot of stats in there. Most of the data is medians so it doesn't tell us much about the outliers which almost certainly are skewing some of the data pretty heavily. There's a lot of mansions in California. The Price per SQ Foot is one way to see a little bit of how the market is shaped. These are broken out by zip code and month.

[Merged Bedrooms](data/zip/merged_bedrooms.csv) - This is also sales data but broken out by the number of bedrooms in the property. That helps us get some sense of how the market is shaped, though of course there are probably $4M one bedrooms skewing things, likely most of those are small condos/townhomes, and we get a little more of a histogram-ish view of the market. These are also broken out by zip code and month though confusingly, in a way that's different than merged sales.

[ca_demographics](data/zip/ca_demographics.csv) - This is 2019 demographic info about the counties in CA. It has tons of info on the county but we need to look up what county a zip code is in for this to be useful. There's examples of that below. What this tends to tell us is "urban/rural", "high income/lower income", really simple categorizations. I might be able to dig up more data by zip code but it's likely to be a bit older. Examples of how to do this are below.

In [95]:
import pandas as pd
import numpy as np

#load dataframe from csv
merged_sales = pd.read_csv("data/zip/merged_sales_stats.csv", delimiter='	')

#print dataframe shape
shape = merged_sales.shape
print('\nDataFrame Shape :', shape)

print(merged_sales.columns)


DataFrame Shape : (137166, 23)
Index(['Unnamed: 0', 'Zip Code', 'Month of Period End', 'Median Sale Price',
       'Median Sale Price MoM ', 'Median Sale Price YoY ', 'Homes Sold',
       'Homes Sold MoM ', 'Homes Sold YoY ', 'New Listings',
       'New Listings MoM ', 'New Listings YoY ', 'Inventory', 'Inventory MoM ',
       ' Inventory YoY ', 'Days on Market', 'Days on Market MoM',
       'Days on Market YoY', 'Average Sale To List',
       'Average Sale To List MoM ', 'Average Sale To List YoY ',
       'record_month', 'PSSF'],
      dtype='object')


The merged_sales_statistics file has all the statistics as columns and zip codes and dates as the rows. I tried to transpose these but it got weird. So let's get some stuff from the merged_sales:

In [16]:
zip90001 = merged_sales[merged_sales['Zip Code'] == 90001] #zip codes are ints
zip90001.head(2)

,Unnamed: 0,Zip Code,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,...,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY,record_month,PSSF
0,0,90001,February 2012,175000,0.0,-7.9,55,-17.9,-6.8,90.0,...,-15.1,-13.5,62.0,27.0,-17.0,95.7,-0.8,-4.5,Zip Code: 900012012-02-01,140.0
1,1,90001,March 2012,165000,-5.7,-8.3,60,9.1,-1.6,92.0,...,-3.3,-5.4,61.0,-1.0,10.0,96.1,0.5,-3.2,Zip Code: 900012012-03-01,145.0


In [94]:
august19 = merged_sales[merged_sales['Month of Period End'] == "August 2019"] #months are strings
august20 = merged_sales[merged_sales['Month of Period End'] == "August 2020"] #months are strings

print(august19['Days on Market'].describe())
print(august20['Days on Market'].describe())

count   1347.0000
mean      48.6630
std       45.2295
min        1.0000
25%       24.0000
50%       41.0000
75%       56.0000
max      654.0000
Name: Days on Market, dtype: float64
count   1338.0000
mean      45.6637
std       46.1157
min        2.0000
25%       22.0000
50%       37.0000
75%       52.0000
max      678.0000
Name: Days on Market, dtype: float64


In [96]:
print(august19['Average Sale To List'].describe())
print(august20['Average Sale To List'].describe())


count   1348.0000
mean      99.4702
std        4.3382
min       78.4000
25%       98.2000
50%       99.3000
75%      100.1000
max      179.0000
Name: Average Sale To List, dtype: float64
count   1339.0000
mean      99.5865
std        2.8212
min       70.3000
25%       98.7000
50%       99.8000
75%      100.5000
max      119.4000
Name: Average Sale To List, dtype: float64


In [21]:


#load dataframe from csv
merged_bedroom_data = pd.read_csv("data/zip/merged_bedrooms.csv", delimiter='	')
print(merged_bedroom_data.shape)
print(merged_bedroom_data.columns)


(1688, 682)
Index(['Unnamed: 0', 'Zip Code', 'June 2009 1bd', 'July 2009 1bd',
       'August 2009 1bd', 'September 2009 1bd', 'October 2009 1bd',
       'November 2009 1bd', 'December 2009 1bd', 'January 2010 1bd',
       ...
       'December 2019 5bd', 'January 2020 5bd', 'February 2020 5bd',
       'March 2020 5bd', 'April 2020 5bd', 'May 2020 5bd', 'June 2020 5bd',
       'July 2020 5bd', 'August 2020 5bd', 'September 2020 5bd'],
      dtype='object', length=682)


In [27]:
# let's get all bedrooms for all zipcodes for August 2020
aug2020_bedrooms = pd.DataFrame(merged_bedroom_data, columns=["Zip Code", "August 2020 1bd", "August 2020 2bd", "August 2020 3bd", "August 2020 4bd", "August 2020 5bd"])

In [28]:
# what's our dataframe look like?
aug2020_bedrooms.head(5)

,Zip Code,August 2020 1bd,August 2020 2bd,August 2020 3bd,August 2020 4bd,August 2020 5bd
0,94109,852358.0,1359790.0,2061146.0,4374846.0,6729337.0
1,90250,495890.0,646922.0,698144.0,782495.0,902262.0
2,94565,227150.0,333446.0,368518.0,514257.0,669362.0
3,90046,595040.0,1058531.0,1919896.0,2830699.0,3621369.0


In [30]:
pd.options.display.float_format = '{:.4f}'.format
# how do we find out things about just 4 bedroom houses in Aug 2020?
aug2020_bedrooms["August 2020 4bd"].describe() 

count      1550.0000
mean     845059.2310
std      738127.1771
min       69945.0000
25%      401323.7500
50%      621232.5000
75%      985655.7500
max     6528913.0000
Name: August 2020 4bd, dtype: float64

In [49]:
''' 
now for our demographic info, it's from 2019 so not super up to date, but pretty close and there's a lot of it
'''
demographics = pd.read_csv("data/zip/ca_demographics.csv", delimiter='	')
zip_to_county = pd.read_csv("data/zip/zip_to_county.csv", delimiter='	')

print(demographics.shape) # a lot of blank columns

for ind in demographics.columns:
    print(ind)

(41, 207)
Geo_FILEID
Geo_STUSAB
Geo_GEOCOMP
Geo_LOGRECNO
Geo_US
Geo_REGION
Geo_DIVISION
Geo_STATECE
Geo_STATE
Geo_COUNTY
Geo_COUSUB
Geo_PLACE
Geo_TRACT
Geo_BLKGRP
Geo_CONCIT
Geo_AIANHH
Geo_AIANHHFP
Geo_AIHHTLI
Geo_AITSCE
Geo_AITS
Geo_ANRC
Geo_CBSA
Geo_CSA
Geo_METDIV
Geo_MACC
Geo_MEMI
Geo_NECTA
Geo_CNECTA
Geo_NECTADIV
Geo_UA
Geo_UACP
Geo_CDCURR
Geo_SLDU
Geo_SLDL
Geo_VTD
Geo_ZCTA3
Geo_ZCTA5
Geo_SUBMCD
Geo_SDELM
Geo_SDSEC
Geo_SDUNI
Geo_UR
Geo_PCI
Geo_TAZ
Geo_UGA
Geo_PUMA5
Geo_PUMA1
Geo_GEOID
Geo_NAME
Geo_BTTR
Geo_BTBG
Geo_FIPS
Geo_QName
SE_A00001_001
SE_A00002_001
SE_A00002_002
SE_A00002_003
SE_A00003_001
SE_A00003_002
SE_A00003_003
SE_A10008_001
SE_A10008_002
SE_A10008_003
SE_A10008_004
SE_A10008_005
SE_A10008_006
SE_A10008_007
SE_A10008_008
SE_A10008_009
SE_A10003_001
SE_A12002_001
SE_A12002_002
SE_A12002_003
SE_A12002_004
SE_A12002_005
SE_A12002_006
SE_A12002_007
SE_A12002_008
SE_A12003_001
SE_A12003_002
SE_A12003_003
SE_A17002_001
SE_A17002_002
SE_A17002_003
SE_A17002_004
SE_A17002_00

Most of the column names are gibberish, so we'll need to use the [ca_demographics_key.txt](data/zip/ca_demographics_key.txt) to make sense of them.

In [37]:
zip_to_county.columns # more straightforward

Index(['Zip Code', 'State', 'County'], dtype='object')

In [84]:
'''
Ok, so let's find out some things about 90001, more or less.
First, which county is it in?
Then, what's that county like?
'''

county90001 = zip_to_county[zip_to_county['Zip Code'] == 90001].iloc[0]['County']
print(county90001)

Los Angeles County


In [85]:

#convoluted, but it works
density = demographics[demographics['Geo_NAME'] == county90001]['SE_A00002_002']
print(density)

10   2473.7873
Name: SE_A00002_002, dtype: float64


So LA is urban. Wow. Mind blown :)
Slightly more interesting: what's the gini co-efficient of LA?

In [93]:
print(demographics[demographics['Geo_NAME'] == county90001].iloc[0]['SE_A14028_001'])

0.494


What percentage of houses are owner occupied?

In [91]:
owner = demographics[demographics['Geo_NAME'] == county90001].iloc[0]['SE_A10060_002']
renter = demographics[demographics['Geo_NAME'] == county90001].iloc[0]['SE_A10060_003']

print(owner / (owner + renter))


0.4541608305256763



We have a lot of this same data for zip codes but it's older. After next week we can bring it in if we think it's worth doing :)